# Scoring

- The maximum number of points for this assignment is 10, the minimum number of points is 0.
- You have two weeks to complete the assignment. Once the assignment is submitted you are not allowed to change it.
- One week delay is penalized with 1 point.
  - Example. The assignment is issued on the 1st January. The deadline without penalization is until 23:59 January 14th (anywhere on Earth). Student A submits his assignment on 22:51 January 13th and is not penalized; student B submits his assignment on 01:13 January 15th and is penalized with 1 point; student C submits his assignment on 3:56 January 23th and is penalized with 2 points and so on.


# Week 6 assignment

- **TASK**
- - In this week we will classify slices from CT scans into two classes: healthy and affected by covid. 
- - We will learn how to project embeddings from the high-dimensional latent space into the plane;
- - We will implement _Expected Calibration Error_ in order to measure how well our model is calibrated;
- - Finally we will perform temperature scaling in order to reduce the _ECE_

- **DATA**
- - We will make use of segmentation data from the week 3;
- - If a slice has less or equal 20 pixels corresponding to covid we will treat this sample as a member of the negative class; if there are more than 20 pixels on the slide then we consider this sample as a member of the positive class;

# Setting up the environment

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Downloading files
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

!wget -nc https://github.com/iburenko/hse_course/raw/main/week3/data/corona_train_segm_dataset.npz
!wget -nc https://github.com/iburenko/hse_course/raw/main/week3/data/corona_val_segm_dataset.npz

zsh:1: command not found: wget
zsh:1: command not found: wget


In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Install required packages into the environment
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

!pip -q install timm


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Import needed packages into the environment
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torchvision import transforms as t
from timm import list_models, create_model

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn import metrics
from sklearn.manifold import TSNE

# Load data into the environment

- **Normalisation**:
In this notebook we will use a different normalisation compared to those that we used in week 3. We will clip outlying values, namely lower than 0.01 and higher than 0.99 percentiles and then we will use 0-1 normalisation, meaning that we subtract from the dataset its minimum value and divide the result of subtraction to the maximum value of the dataset;
- Feel free to change the normalisation if you find it appropriate.

In [2]:
train_data = dict(np.load('../week3/data/corona_train_segm_dataset.npz', allow_pickle=True))
val_data = dict(np.load('../week3/data/corona_val_segm_dataset.npz', allow_pickle=True))

In [3]:
lower_bound = np.quantile(train_data['images'], 0.01)
upper_bound = np.quantile(train_data['images'], 0.99)

# Define Dataset class

- In this variant we use some random augmentations to transform the training data;
- No augmentation is used for the validation dataset;
- Feel free to change the set of augmentations.

In [4]:
class SegmentationDatasetNumpy(Dataset):
    def __init__(self, train, lower_bound, upper_bound):
        super().__init__()
        assert train in ['train', 'val']
        self.train = train
        if self.train == 'train':
            data = train_data
            data_transform = t.Compose(
              [
                  t.RandomRotation(15),
                  t.RandomHorizontalFlip(),
                  t.RandomVerticalFlip(),
                  t.GaussianBlur(3),
              ]
          )
        else:
            data = val_data
            data_transform = t.Compose([])
        self.all_data = data['images']
        self.all_data = self._normalise_data(
            self.all_data, lower_bound, upper_bound
            )
        self.all_masks = (data['masks'].sum(axis=(0,1)) > 20)
        self.data_transform = data_transform

    def __len__(self):
        return self.all_data.shape[-1]
    
    def __getitem__(self, idx):
        return self.all_data[...,idx], self.all_masks[...,idx]

    def _normalise_data(self, data, lower_bound, upper_bound):
      data = np.clip(data, lower_bound, upper_bound)
      data = (data - lower_bound)/(upper_bound-lower_bound)
      return data

    def collate(self, batch):
      x = np.stack([elem[0] for elem in batch])
      y = np.stack([elem[1] for elem in batch]).astype('float32')
      x = self.data_transform(torch.from_numpy(x).unsqueeze(1))
      y = torch.from_numpy(y)
      return x, y

# Initialise Dataset and DataLoader classes

In [ ]:
train_dataset = SegmentationDatasetNumpy('train', lower_bound, upper_bound)
val_dataset = SegmentationDatasetNumpy('val', lower_bound, upper_bound)

In [ ]:
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=16, 
    shuffle=True, 
    num_workers=0,
    collate_fn = train_dataset.collate
    )

val_dataloader = DataLoader(
    val_dataset, 
    batch_size=16, 
    shuffle=False, 
    num_workers=0,
    collate_fn = val_dataset.collate
    )

# Define a model

- You could use a model from the ```timm``` library. By default we will use ```resnet50```. You can change it to your liking.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = create_model(
    'resnet50', 
    pretrained=False, 
    in_chans=1, 
    num_classes=1
    ).to(device)

# TASK 1.
**(1 point)**


Define a feature extractor:
```feature_extractor = ...```

- Feature extractor is a model that processes the input image and outputs an embedding vector. The embedding vector is not a vector of logits;
- In most of cases after the last convolutional layer one applies some pooling layer and obtains the embeddings (it might be the case that after pooling linear representation processed by some MLP);
- In any case if the vector of logits is the output vector of the model, the embeddings vector is extracted from the pre-last (in most of cases) layer;
- **Hint**: Use ```model.children()``` or similar methods.

In [ ]:
# feature_extractor = ...

# Embeddings of randomly initialized network
**(1 point)**

- Get the embeddings of the randomly intialized network.

In [ ]:
_ = model.eval()

# embeddings = ...

# Define train/val loops

In [ ]:
def train(model):  
  model = model.train()
  train_loss = 0
  train_dice = 0
  for batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    x, y = batch
    pred = model(x.to(device)).squeeze()
    pred = pred.cpu()
    loss = criterion(pred, y)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
  return train_loss

def val(model):
  model = model.eval()
  val_loss, val_acc = 0, 0
  for batch in tqdm(val_dataloader):
    x, y = batch
    with torch.no_grad():
      pred = model(x.to(device)).squeeze()
    pred = pred.cpu()
    loss = criterion(pred, y.float())
    val_loss += loss.item()
    val_acc += ((pred>0).float() == y).sum()
  return val_loss, val_acc

# Define a loss function and optimizer


In [ ]:
criterion = BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=3e-4, weight_decay=1e-5)

# Train a binary classifier

In [ ]:
epochs = 10

for epoch in range(epochs):
  train_loss = train(model)
  val_loss, val_acc = val(model)
  train_loss = train_loss/len(train_dataset)
  val_loss = val_loss/len(val_dataset)
  val_acc = val_acc/len(val_dataset)
  print(f'Train loss = {train_loss:.3f} after the {epoch:d} epochs')
  print(f'Val loss = {val_loss:.3f} after the {epoch:d} epochs')
  print(f'Val accuracy = {val_acc:.3f} after the {epoch:d} epochs')
  print('='*80)

# Compare t-SNE visualisations of the trained and randomly initialised networks

**(1 point)**

---


Plot the t-SNE embeddings of the trained network and compare them to the visualisation of the untrained network.

In [ ]:
# tsne_random = ...
# tsne_trained = ...
# Your visualisation here

#TASK 2

# Expected Calibration Error

- We will divide the probabilities into bins;
- We will store bins in a dictionary; a key will correspond to the bin's number and a value will be locations of items in a tensor. Using this value we can access needed values simply using
```probs[inds]```

---

# Calculate confidences and ECE
- **(1 point)** Write the function that calculates the confidence for each bin. Recall that confidence is the average probability of items in a bin;
- Return a ```dict``` (similar to ```calculate_bin_accuracy```);
- **(1 point)** Write a function that calculates ECE. ECE is the averaged absolute difference between accuracies and confidences;
- Mind ```nan```'s!
- ECE calculation should work correctly for any number of bins!

In [ ]:
def bin_inds(probs, y, num_bins):
  bins = dict()
  thrs = np.linspace(0, 1, num_bins+1)
  for i in range(len(thrs)-1):
    bin_locations = np.logical_and(
            probs >= thrs[i], probs < thrs[i+1]
        )
    bins[i] = bin_locations
  return bins

def calculate_bin_accuracy(pred, y, bins):
  bin_accs = dict()
  for key, inds in bins.items():
    acc = metrics.accuracy_score(
        y[inds], pred[inds]>=0.5
    )
    if np.isnan(acc):
      bin_accs[key] = 0
    else:
      bin_accs[key] = acc
  return bin_accs

def calculate_bin_confidence(pred, bins):
  bin_confs = dict()
  # Your code here
  return bin_confs

def calculate_ece(bin_accs, bin_confs, bins, y):
  ece = 0
  # Your code here
  return ece

# Finding temperature
**(2 point)**

We will find an optimal value of the temperature using SGD. For that we need to define a set of parameters that we will optimize.

---

- Define a variable $\tau$ as a pytorch's parameter;
- Use this parameter to initialize optimizer;

In [ ]:
# Define tau as a parameter
# tau = ...
# Define an optimizer
# optimizer = ...

# Train loop for the temperature scaling

**(2 points)**

Write a train loop that performs gradient descent and finds the optimal value of the parameter $\tau$.

---

Hints:
- Use cross-entropy loss (in this binary case you could use BCEWithLogits);
- Do not forget that you do not need to optimize model's parameters;

In [ ]:
def train_ece(model):  
  train_loss = 0
  model = model.eval()
  # Your code here
  
  return train_loss


# Find the optimal value of $\tau$.

In [ ]:
epochs = 50

for _ in tqdm(range(epochs)):
  train_ece(model)

print(tau)

# Compare ECEs
**(1 point)**

---

- Calculate ECE for $\tau = 1$ (uncalibrated);
- What value of $\tau$ did you obtain?
- Calculate ECE using the value of $\tau$ obtained after temperature scaling;
- Compare results.

In [ ]:
def get_val_logits(model):
  preds = torch.zeros(len(val_dataset))
  for i, batch in enumerate(val_dataloader):
    x,y = batch
    with torch.no_grad():
      pred = model(x.to(device)).squeeze()
    preds[16*i:16*(i+1)] = pred.cpu()
  return preds

def get_val_probs(logits, tau):
  logits = logits/tau
  return torch.sigmoid(logits)

num_bins = 10

# Your comparison here